In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [38]:
ratings_df = pd.read_csv('train2.csv')

In [39]:
ratings_df = ratings_df[['user_id','movie_id','rating']]
ratings_df.columns= ['UserID','MovieID','Rating']
ratings_df

,UserID,MovieID,Rating
0,173,289,4
1,607,174,3
2,600,53,4
3,104,456,3
4,486,123,3
...,...,...,...
79995,683,301,2
79996,191,750,4
79997,625,169,5
79998,128,402,1


In [40]:
movies_df = pd.read_csv('train2.csv')
movies_df = movies_df[['movie_id','movie title']]
movies_df.columns = ['MovieID','Title']

In [52]:
movies_df =movies_df.drop_duplicates(keep='first')

In [53]:
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,1669,1670,1671,1672,1674,1677,1678,1680,1681,1682
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
R = R_df.values
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [55]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [56]:
sigma = np.diag(sigma)

In [57]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [58]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
preds_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,1669,1670,1671,1672,1674,1677,1678,1680,1681,1682
0,5.843864,2.048294,2.467900,3.482916,1.229875,1.051912,3.325330,1.908131,1.759193,2.686086,...,0.009084,0.007124,0.008623,0.040767,-0.066648,0.037521,0.011043,0.009079,0.080798,0.094251
1,2.419105,0.545902,-0.195571,-0.105873,-0.100680,0.126591,0.172476,0.095744,1.265786,0.237687,...,0.001927,0.010864,-0.015001,-0.013271,-0.020664,0.007351,-0.006166,0.003616,0.015788,-0.080826
2,0.137337,-0.049103,0.037603,-0.097599,0.157764,-0.141077,-0.033816,0.236197,-0.227123,0.365418,...,0.009940,0.020266,-0.004340,-0.007883,-0.023011,-0.026368,0.003254,0.017221,-0.014326,-0.016910
3,-0.095988,-0.047384,0.186420,-0.267959,0.249450,-0.058952,0.080500,0.023147,0.199376,-0.020360,...,0.016178,0.018656,0.012946,0.007646,0.020630,0.024957,0.021546,0.031871,-0.000818,-0.000937
4,2.920610,0.977254,0.154194,0.409062,0.309873,-0.035703,2.443146,0.717780,-0.164904,0.228736,...,0.041340,0.040510,0.042462,0.046521,0.078440,0.028750,0.042944,0.042889,0.015367,0.042622


In [63]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=10):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return  recommendations

In [60]:
rated, predictions = recommend_movies(preds_df, 837, movies_df, ratings_df, 10)

User 837 has already rated 38 movies.
Recommending highest 10 predicted ratings movies not already rated.


In [61]:
rated

,UserID,MovieID,Rating,Title
22,837,151,5,Willy Wonka and the Chocolate Factory (1971)
21,837,740,5,Jane Eyre (1996)
3,837,289,5,Evita (1996)
28,837,1009,5,Stealing Beauty (1996)
29,837,283,5,Emma (1996)
10,837,125,5,Phenomenon (1996)
13,837,275,4,Sense and Sensibility (1995)
20,837,111,4,"Truth About Cats & Dogs, The (1996)"
18,837,294,4,Liar Liar (1997)
17,837,13,4,Mighty Aphrodite (1995)


In [62]:
predictions

,MovieID,Title
454,742,Ransom (1996)
604,14,"Postino, Il (1994)"
318,508,"People vs. Larry Flynt, The (1996)"
363,127,"Godfather, The (1972)"
441,15,Mr. Holland's Opus (1995)
267,1,Toy Story (1995)
30,50,Star Wars (1977)
45,471,Courage Under Fire (1996)
50,286,"English Patient, The (1996)"
537,116,Cold Comfort Farm (1995)


In [65]:
rec = pd.read_csv('recommendations.csv', header =None)

In [66]:
value = []
for i in rec[0].values.tolist():
    final = recommend_movies(preds_df, i, movies_df, ratings_df, 10)
    pred = final['MovieID'].values.tolist()
    value.append(pred)

User 1 has already rated 228 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 2 has already rated 51 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 3 has already rated 47 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 4 has already rated 20 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 5 has already rated 139 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 6 has already rated 168 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 7 has already rated 336 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 8 has already rated 47 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 9 has already rated 15 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 10 has already rated 138 movies.
Recommending highest 1

User 89 has already rated 57 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 90 has already rated 244 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 91 has already rated 76 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 92 has already rated 302 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 93 has already rated 18 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 94 has already rated 329 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 95 has already rated 232 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 96 has already rated 48 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 97 has already rated 50 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 98 has already rated 20 movies.
Recommending h

User 169 has already rated 33 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 170 has already rated 18 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 171 has already rated 22 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 172 has already rated 24 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 173 has already rated 32 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 174 has already rated 130 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 175 has already rated 33 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 176 has already rated 48 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 177 has already rated 85 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 178 has already rated 219 movies.
Recomm

User 269 has already rated 256 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 270 has already rated 103 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 271 has already rated 223 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 272 has already rated 34 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 273 has already rated 17 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 274 has already rated 54 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 275 has already rated 78 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 276 has already rated 408 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 277 has already rated 42 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 278 has already rated 17 movies.
Reco

User 364 has already rated 18 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 365 has already rated 47 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 366 has already rated 21 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 367 has already rated 44 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 368 has already rated 35 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 369 has already rated 19 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 370 has already rated 60 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 371 has already rated 38 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 372 has already rated 53 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 373 has already rated 192 movies.
Recomme

...

User 520 has already rated 19 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 521 has already rated 120 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 522 has already rated 26 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 523 has already rated 76 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 524 has already rated 241 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 525 has already rated 45 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 526 has already rated 50 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 527 has already rated 109 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 528 has already rated 41 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 529 has already rated 36 movies.
Recom

User 601 has already rated 119 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 602 has already rated 26 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 603 has already rated 39 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 604 has already rated 21 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 605 has already rated 61 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 606 has already rated 175 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 607 has already rated 29 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 608 has already rated 140 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 609 has already rated 25 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 610 has already rated 59 movies.
Recom

User 680 has already rated 37 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 681 has already rated 17 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 682 has already rated 318 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 683 has already rated 59 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 684 has already rated 72 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 685 has already rated 17 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 686 has already rated 51 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 687 has already rated 15 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 688 has already rated 20 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 689 has already rated 26 movies.
Recomme

Recommending highest 10 predicted ratings movies not already rated.
User 778 has already rated 54 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 779 has already rated 30 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 780 has already rated 45 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 781 has already rated 38 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 782 has already rated 198 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 783 has already rated 25 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 784 has already rated 33 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 785 has already rated 24 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 786 has already rated 100 movies.
Recommending highest 10 predicted ratings mo

User 880 has already rated 300 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 881 has already rated 216 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 882 has already rated 106 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 883 has already rated 213 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 884 has already rated 33 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 885 has already rated 78 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 886 has already rated 202 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 887 has already rated 142 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 888 has already rated 17 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 889 has already rated 253 movies.
R

In [67]:
value

[[408, 196, 385, 475, 511, 432, 28, 154, 168, 692],
 [137, 124, 15, 181, 286, 750, 248, 25, 316, 321],
 [313, 268, 271, 301, 315, 748, 310, 678, 690, 270],
 [269, 302, 181, 268, 300, 321, 294, 272, 56, 328],
 [153, 175, 7, 265, 431, 229, 28, 655, 269, 588],
 [603, 484, 192, 527, 654, 663, 657, 28, 150, 661],
 [183, 136, 91, 184, 498, 491, 508, 199, 525, 131],
 [174, 300, 89, 230, 403, 226, 161, 62, 117, 173],
 [185, 9, 302, 134, 480, 288, 127, 132, 237, 748],
 [23, 89, 8, 194, 14, 55, 187, 209, 15, 162],
 [209, 568, 328, 151, 70, 380, 655, 153, 98, 13],
 [210, 286, 427, 186, 153, 95, 173, 70, 69, 64],
 [529, 304, 270, 430, 274, 347, 399, 94, 15, 1041],
 [174, 285, 286, 11, 209, 182, 137, 1, 169, 204],
 [304, 118, 14, 597, 242, 111, 284, 281, 126, 321],
 [95, 69, 238, 82, 222, 124, 218, 23, 568, 156],
 [7, 268, 14, 25, 300, 288, 9, 50, 15, 124],
 [174, 135, 511, 275, 521, 70, 523, 192, 178, 485],
 [300, 321, 204, 70, 168, 194, 191, 527, 173, 95],
 [117, 79, 275, 222, 125, 82, 195, 64, 1

In [68]:
value1 = pd.DataFrame(value)

In [69]:
value1.index = rec[0].values.tolist()

In [71]:
value1.to_csv('recommendations500.csv', header = None)